# Introduction 

In this notebook, we will implement [*Latent Credible Analysis*](https://research.fb.com/publications/latent-credibility-analysis/) models. These are latent probablistic models that use hidden (latent) variables to represents the unknown data source reliabilities and underlying truth values. 

We implement only simpleLCA for now as extension to other models are relatively straight forward.

# SimpleLCA

Here is the plate model of simpleLCA. 

![simpleLCA](./gfx/simpleLCA.png)

### Data 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os.path as op
import numpy as np
import seaborn as sns
import pyro

In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from spectrum.preprocessing import encoders
from spectrum.discovers import lca

In [ ]:
DATA_DIR = '../data'
DATA_SET = 'population'

In [ ]:
truths = pd.read_csv(op.join(DATA_DIR, DATA_SET, 'truths.csv'))
claims = pd.read_csv(op.join(DATA_DIR, DATA_SET, 'claims.csv'))

In [ ]:
truths.head()

In [ ]:
claims.head()

In [ ]:
truths.shape, claims.shape

We decide to model city population as discrete value. Moreover we consider the hidden truth value is only from the set of available assertions. Thus we need to label encode `value` of claims data frame.

### Data Preprocessing 

We need to label encode values of objects in order to feed them to our simpleLCA model

In [ ]:
claims_enc, le_dict = encoders.transform(claims)

build the confidence matrix, $[w_{s,o}]$ in the paper, if $w_{s,o} = 1$, then the source s does make an assertion about object o.

In [ ]:
W = lca.build_mask(claims)

In [ ]:
W.shape, claims.source_id.nunique(), claims.object_id.nunique()

we also need to build an observation dictionary.

In [ ]:
claims.head()

In [ ]:
claims_enc.head()

In [ ]:
observation = lca.build_observation(claims_enc)

In [ ]:
# claims.groupby(['object_id']).nunique()

### Model

Create some data

In [ ]:
claims = dict()
claims['source_id'] = [0, 0, 1]
claims['object_id'] = [0, 1, 1]
claims['value'] = [0, 1, 0]
claims = pd.DataFrame(data=claims)

build inputs for simpleLCA model

In [ ]:
mask = lca.build_mask(claims)
observation = lca.build_observation(claims)

In [ ]:
def generate_one_simpleLCA_sample(observation, mask):
    tracer = pyro.poutine.trace(lca.lca_model)
    trace = tracer.get_trace(observation, mask)

    for name, node in trace.nodes.items():
        if node['type'] == 'sample':
            print(f'{node["name"]} - sampled value {node["value"]} ')

In [ ]:
for i in range(3):
    generate_one_simpleLCA_sample(observation, mask)
    print('-'*10)

# Inference